# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [34]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [35]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [36]:
# Combine all features
X_train = pd.merge(X_train_numeric, X_train_cat, left_index=True, right_index=True)
X_val = pd.merge(X_val_numeric,X_val_cat, left_index=True, right_index=True)
X_test = pd.merge(X_test_numeric, X_test_cat, left_index=True, right_index=True)

As a refresher, preview the training data: 

In [37]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [38]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [39]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100,activation='relu', input_shape=n_features))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [40]:
# Numeric column names
numeric_columns = X_test_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [41]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [42]:
# Train the model
normalized_input_model.fit(X_train,
                            y_train, 
                            batch_size=32,
                            epochs=150,
                            validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 1035246698894625657875070976.0000 - mse: 1035246698894625657875070976.0000 - val_loss: 312868308705160063877120.0000 - val_mse: 312868308705160063877120.0000
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 176739425906114330361856.0000 - mse: 176739425906114330361856.0000 - val_loss: 82467403367816450015232.0000 - val_mse: 82467403367816450015232.0000
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 46585883063866751975424.0000 - mse: 46585883063866751975424.0000 - val_loss: 21737182047857492885504.0000 - val_mse: 21737182047857492885504.0000
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 12279346984902274842624.0000 - mse: 12279346984902274842624.0000 - val_loss: 5729598791330870329344.0000 - val_mse: 5729598791330870329344.0000
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 3236654017073045831680.0000 - mse: 3236

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [43]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [44]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.5168 - mse: 0.5168 - val_loss: 0.2646 - val_mse: 0.2646
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2301 - mse: 0.2301 - val_loss: 0.1871 - val_mse: 0.1871
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1902 - mse: 0.1902 - val_loss: 0.1724 - val_mse: 0.1724
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1663 - mse: 0.1663 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1525 - mse: 0.1525 - val_loss: 0.1542 - val_mse: 0.1542
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1411 - mse: 0.1411 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1275 - mse: 0.1275 - val_loss: 0.1435 - val_mse: 0.1435
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1243 - m

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [45]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0088 - mse: 0.0088


[0.008778097108006477, 0.008778097108006477]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [46]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1441 - mse: 0.1441


[0.14410679042339325, 0.14410679042339325]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [47]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

29831.40560039921

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [48]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])
# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 0.4902 - mse: 0.4902 - val_loss: 0.2117 - val_mse: 0.2117
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2672 - mse: 0.2672 - val_loss: 0.1575 - val_mse: 0.1575
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.1548 - val_mse: 0.1548
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1774 - mse: 0.1774 - val_loss: 0.1675 - val_mse: 0.1675
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1518 - mse: 0.1518 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1360 - mse: 0.1360 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1294 - mse: 0.1294 - val_loss: 0.1244 - val_mse: 0.1244
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1177 - m

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [49]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0067 - mse: 0.0067


[0.006720090284943581, 0.006720090284943581]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [50]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 2ms/step - loss: 0.1490 - mse: 0.1490


[0.1489548534154892, 0.1489548534154892]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [51]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 14ms/step - loss: 0.3859 - mse: 0.3859 - val_loss: 0.1564 - val_mse: 0.1564
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2290 - mse: 0.2290 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1781 - mse: 0.1781 - val_loss: 0.1317 - val_mse: 0.1317
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1534 - mse: 0.1534 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1341 - mse: 0.1341 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1169 - mse: 0.1169 - val_loss: 0.1090 - val_mse: 0.1090
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1082 - mse: 0.1082 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1016 - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [52]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0076 - mse: 0.0076


[0.007634198293089867, 0.007634198293089867]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [53]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1273 - mse: 0.1273


[0.12728245556354523, 0.12728245556354523]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [54]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', loss = 'mse', metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 13ms/step - loss: 0.3211 - mse: 0.3211 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1726 - mse: 0.1726 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1217 - mse: 0.1217 - val_loss: 0.0951 - val_mse: 0.0951
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0965 - mse: 0.0965 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0665 - mse: 0.0665 - val_loss: 0.1558 - val_mse: 0.1558
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0626 - mse: 0.0626 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0510 - mse: 0.0510 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0399 - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [55]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0044 - mse: 0.0044


[0.004399280063807964, 0.004399280063807964]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [56]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1016 - mse: 0.1016


[0.10161010921001434, 0.10161010921001434]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [57]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='adam', loss = 'mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 20ms/step - loss: 0.3768 - mse: 0.3768 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1595 - mse: 0.1595 - val_loss: 0.1335 - val_mse: 0.1335
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1078 - mse: 0.1078 - val_loss: 0.1585 - val_mse: 0.1585
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0830 - mse: 0.0830 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0587 - mse: 0.0587 - val_loss: 0.1385 - val_mse: 0.1385
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0471 - mse: 0.0471 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0374 - mse: 0.0374 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0294 - 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [58]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0079 - mse: 0.0079


[0.007921255193650723, 0.007921255193650723]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [59]:
# Evaluate the model on validate data
adam_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1075 - mse: 0.1075


[0.10750605165958405, 0.10750605165958405]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [60]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.1431 - mse: 0.1431


[0.14306391775608063, 0.14306391775608063]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [61]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred_scaled))

206343.50370476366

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.